<a href="https://colab.research.google.com/github/StevenVuong/compvis_playground/blob/master/jupyter_notebooks/Image_Super_Resolution_Prediction_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/idealo/image-super-resolution/blob/master/notebooks/ISR_Prediction_Tutorial.ipynb)

Credits to Idealo. References & Further Documentation:
-  https://github.com/idealo/image-super-resolution 
-  https://idealo.github.io/image-super-resolution/tutorials/docker/ 

### Fuse To GCS

In [0]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  22517      0 --:--:-- --:--:-- --:--:-- 23321
OK
25 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 4,274 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.28.1_amd64.deb ...
Unpacking gcsfuse (0.28.1) ...
Setting up gcsfuse (0.28.1) ...


In [7]:
!mkdir compvis_playground
!gcsfuse --implicit-dirs compvis_playground /content/compvis_playground

Using mount point: /content/compvis_playground2
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [0]:
!ls compvis_playground/facenet

# Install ISR
(skip if using local repo)

In [0]:
!git clone https://github.com/idealo/image-super-resolution
%cd image-super-resolution
!python3 ./setup.py install

# Predict

Download a sample image

In [0]:
!wget http://images.math.cnrs.fr/IMG/png/section8-image.png
!mkdir -p data/input/test_images
!mv *.png data/input/test_images

Load the image with PIL

In [0]:
import numpy as np
from PIL import Image

img = Image.open('data/input/test_images/section8-image.png')
img

## Get predictions

### Create the model and run prediction
Create a RRDN or RDN model with the provided pre-trained weights, choose one

In [0]:
# Uncomment if using local repo
# import sys
# sys.path.append('..')
from ISR.models import RDN, RRDN

# model = RDN(weights='noise-cancel')
# model = RRDN(weights='gans')
model = RDN(weights='psnr-small')
# model = RDN(weights='psnr-large')

#### Alternatively, load your own weights
To load your own weights first build the model specifying the architecture parameters and then load the weights like with any Keras model.

In [0]:
# rdn = RDN(arch_params={'C': 5, 'D':16, 'G':48, 'G0':52, 'x':3})
# rdn.model.load_weights('PATH/TO/WEIGHTS')

#### Baseline

In [0]:
img.resize(size=(img.size[0]*4, img.size[1]*4), resample=Image.BICUBIC)

#### Prediction

In [0]:
sr_img = model.predict(np.array(img))
Image.fromarray(sr_img)

### Usecase: upscaling noisy images

Compress the image into the jpeg format to introduce compression artefact and lose some information.

In [0]:
img.save('data/input/test_images/compressed.jpeg','JPEG', dpi=[300, 300], quality=50)
compressed_img = Image.open('data/input/test_images/compressed.jpeg')

compressed_img

#### Baseline

In [0]:
compressed_img.resize(size=(compressed_img.size[0]*2, compressed_img.size[1]*2), resample=Image.BICUBIC)

In [0]:
model = RDN(weights='noise-cancel')
sr_img = model.predict(np.array(compressed_img))
Image.fromarray(sr_img)

### Predictor Class
You can also use the predictor class to run the model on entire folders:

In [0]:
from ISR.predict import Predictor
!mkdir -p data/output
predictor = Predictor(input_dir='data/input/sample/')
predictor.get_predictions(model=model, weights_path='weights/rdn-C6-D20-G64-G064-x2_ArtefactCancelling_epoch219.hdf5')